# Importing Sampled Stratified csv File

In [ ]:
from pandas import read_csv
import pandas as pd

# Specify the encoding as detected ('ascii')
sampled_data = pd.read_csv('/content/Simple_Random_Sample.csv', encoding='ascii')  # Updated encoding to 'ascii'

# Data Preprocessing

## Splitting into Training and Test Set using Sampled Dataset

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'sampled_data' is your dataframe after sampling 20,000 rows

# Split into train and test sets (e.g., 70% train, 30% test)
train_data, test_data = train_test_split(sampled_data, test_size=0.3, random_state=42, shuffle=True)

print(f"Training set size: {train_data.shape}")
print(f"Test set size: {test_data.shape}")

Training set size: (21000, 78)
Test set size: (9000, 78)


## Feature Engineering

In [ ]:
#############################

# Feature: Discount Sensitivity
train_data['discount_sensitivity'] = train_data['total_discounts_received'] / train_data['total_items_purchased']
test_data['discount_sensitivity'] = test_data['total_discounts_received'] / test_data['total_items_purchased']

print(train_data[['customer_id', 'discount_sensitivity']].head())
'''
Indicates how much discount (in dollars) a customer receives per item purchased on average.
'''

#############################

# Feature: Average Discount Per Transaction
train_data['avg_discount_per_transaction'] = train_data['total_discounts_received'] / train_data['total_transactions']
test_data['avg_discount_per_transaction'] = test_data['total_discounts_received'] / test_data['total_transactions']

print(train_data[['customer_id', 'avg_discount_per_transaction']].head())
'''
Indicates how much discount, on average, a customer gets per transaction.
Useful for analyzing price sensitivity and promotional effectiveness.
'''

#############################

# Feature: Average Transaction Value
train_data['avg_transaction_value'] = train_data['total_sales'] / train_data['total_transactions']
test_data['avg_transaction_value'] = test_data['total_sales'] / test_data['total_transactions']

print(train_data[['customer_id', 'avg_transaction_value']].head())
'''
Provides insights into how much a customer spends on average per transaction.
Helps identify valuable customers.
'''

#############################

# Feature: High Value Purchase (based on avg_purchase_value)
high_value_threshold = train_data['avg_purchase_value'].quantile(0.75)

train_data['high_value_purchase'] = train_data['avg_purchase_value'] > high_value_threshold
test_data['high_value_purchase'] = test_data['avg_purchase_value'] > high_value_threshold

print(train_data[['customer_id', 'avg_purchase_value', 'high_value_purchase']].head())
'''
Customers with average purchase value above 75th percentile are marked as high-value purchasers.
'''

#############################

# Feature: High Value Quantity (based on quantity)
quantity_threshold = train_data['quantity'].quantile(0.75)

train_data['high_value_quantity'] = train_data['quantity'] > quantity_threshold
test_data['high_value_quantity'] = test_data['quantity'] > quantity_threshold

print(train_data[['customer_id', 'quantity', 'high_value_quantity']].head())
'''
Flags transactions where quantity is above 75th percentile.
Can identify bulk buyers.
'''

#############################

# Feature: Customer Lifetime Value (CLV)
train_data['CLV'] = train_data.groupby('customer_id')['total_sales'].transform('sum')
test_data['CLV'] = test_data.groupby('customer_id')['total_sales'].transform('sum')

print(train_data[['customer_id', 'CLV']].head())
'''
Total revenue a customer generates across their lifetime.
Useful for prioritizing marketing efforts.
'''

#############################

# Feature: Total Return Rate
train_data['total_return_rate'] = train_data['total_returned_items'] / train_data['total_items_purchased']
test_data['total_return_rate'] = test_data['total_returned_items'] / test_data['total_items_purchased']

print(train_data[['customer_id', 'total_return_rate']].head())
'''
Percentage of items returned vs purchased.
High return rate may indicate dissatisfaction or quality issues.
'''


       customer_id  discount_sensitivity
28465       928854             12.415472
27622       985883              1.391952
28376       894713              2.087427
10917       668403              2.292376
27234       747173              0.194110
       customer_id  avg_discount_per_transaction
28465       928854                     14.304783
27622       985883                     15.650976
28376       894713                     11.154687
10917       668403                     17.559600
27234       747173                      1.790943
       customer_id  avg_transaction_value
28465       928854               4.761522
27622       985883             130.382927
28376       894713              99.018281
10917       668403             105.153200
27234       747173             107.606604
       customer_id  avg_purchase_value  high_value_purchase
28465       928854              389.40                 True
27622       985883              276.95                False
28376       894713          

'\nPercentage of items returned vs purchased.\nHigh return rate may indicate dissatisfaction or quality issues.\n'

## Date Binning / Feature Extraction from Datetime Data

In [ ]:
# Date Binning/Feature Extraction for train_data
train_data['transaction_month'] = pd.to_datetime(train_data['transaction_date']).dt.month
train_data['transaction_year'] = pd.to_datetime(train_data['transaction_date']).dt.year
# Extracts the month and year from the transaction date for trend analysis

train_data['product_expiry_date_month'] = pd.to_datetime(train_data['product_expiry_date']).dt.month
train_data['product_expiry_date_year'] = pd.to_datetime(train_data['product_expiry_date']).dt.year
# Converts product expiry date into month and year for expiration trends

train_data['product_manufacture_month'] = pd.to_datetime(train_data['product_manufacture_date']).dt.month
train_data['product_manufacture_year'] = pd.to_datetime(train_data['product_manufacture_date']).dt.year
# Extracts manufacturing month and year for production timeline analysis

train_data['promotion_end_month'] = pd.to_datetime(train_data['promotion_end_date']).dt.month
train_data['promotion_end_year'] = pd.to_datetime(train_data['promotion_end_date']).dt.year
# Converts promotion end date into month and year for promotion end analysis

train_data['last_purchase_month'] = pd.to_datetime(train_data['last_purchase_date']).dt.month
train_data['last_purchase_year'] = pd.to_datetime(train_data['last_purchase_date']).dt.year
# Extracts last purchase month and year for customer purchase behavior

train_data['promotion_start_month'] = pd.to_datetime(train_data['promotion_start_date']).dt.month
train_data['promotion_start_year'] = pd.to_datetime(train_data['promotion_start_date']).dt.year
# Converts promotion start date into month and year for promotional activity timing

# Date Binning/Feature Extraction for test_data
test_data['transaction_month'] = pd.to_datetime(test_data['transaction_date']).dt.month
test_data['transaction_year'] = pd.to_datetime(test_data['transaction_date']).dt.year
# Extracts the month and year from the transaction date for trend analysis

test_data['product_expiry_date_month'] = pd.to_datetime(test_data['product_expiry_date']).dt.month
test_data['product_expiry_date_year'] = pd.to_datetime(test_data['product_expiry_date']).dt.year
# Converts product expiry date into month and year for expiration trends

test_data['product_manufacture_month'] = pd.to_datetime(test_data['product_manufacture_date']).dt.month
test_data['product_manufacture_year'] = pd.to_datetime(test_data['product_manufacture_date']).dt.year
# Extracts manufacturing month and year for production timeline analysis

test_data['promotion_end_month'] = pd.to_datetime(test_data['promotion_end_date']).dt.month
test_data['promotion_end_year'] = pd.to_datetime(test_data['promotion_end_date']).dt.year
# Converts promotion end date into month and year for promotion end analysis

test_data['last_purchase_month'] = pd.to_datetime(test_data['last_purchase_date']).dt.month
test_data['last_purchase_year'] = pd.to_datetime(test_data['last_purchase_date']).dt.year
# Extracts last purchase month and year for customer purchase behavior

test_data['promotion_start_month'] = pd.to_datetime(test_data['promotion_start_date']).dt.month
test_data['promotion_start_year'] = pd.to_datetime(test_data['promotion_start_date']).dt.year
# Converts promotion start date into month and year for promotional activity timing

# Define columns to drop
date_columns_to_drop = [
    'transaction_date',
    'product_expiry_date',
    'product_manufacture_date',
    'promotion_end_date',
    'last_purchase_date',
    'promotion_start_date'
]

# Drop original date columns from train_data and test_data
train_data = train_data.drop(columns=date_columns_to_drop)
test_data = test_data.drop(columns=date_columns_to_drop)

# Verify columns after dropping original dates
print("Train data columns after dropping original dates:")
print(train_data.columns.tolist())
print("\nTest data columns after dropping original dates:")
print(test_data.columns.tolist())

Train data columns after dropping original dates:
['customer_id', 'age', 'gender', 'income_bracket', 'loyalty_program', 'membership_years', 'churned', 'marital_status', 'number_of_children', 'education_level', 'occupation', 'transaction_id', 'product_id', 'product_category', 'quantity', 'unit_price', 'discount_applied', 'payment_method', 'store_location', 'transaction_hour', 'day_of_week', 'week_of_year', 'month_of_year', 'avg_purchase_value', 'purchase_frequency', 'avg_discount_used', 'preferred_store', 'online_purchases', 'in_store_purchases', 'avg_items_per_transaction', 'avg_transaction_value', 'total_returned_items', 'total_returned_value', 'total_sales', 'total_transactions', 'total_items_purchased', 'total_discounts_received', 'avg_spent_per_category', 'max_single_purchase_value', 'min_single_purchase_value', 'product_name', 'product_brand', 'product_rating', 'product_review_count', 'product_stock', 'product_return_rate', 'product_size', 'product_weight', 'product_color', 'produ

## Unusual Outliers Check

In [ ]:
import pandas as pd
import numpy as np

# Set pandas options to display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Function to detect outliers using IQR
def detect_outliers_iqr(data):
    numerical_cols = data.select_dtypes(include=np.number).columns
    outliers_iqr = {}

    for col in numerical_cols:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)][col]
        outliers_iqr[col] = {
            'count': len(outliers),
            'percentage': (len(outliers) / len(data)) * 100,
            'outlier_values': outliers.values
        }

    return outliers_iqr

#############################

# Detect and print outliers in TRAINING SET
print("\n=== Outliers (IQR Method) – Train Data ===")
outliers_iqr_train = detect_outliers_iqr(train_data)

for col, info in outliers_iqr_train.items():
    print(f"\nColumn: {col}")
    print(f"Number of outliers: {info['count']}")
    print(f"Percentage of outliers: {info['percentage']:.2f}%")
    if info['count'] > 0:
        print(f"Outlier values: {info['outlier_values'][:10]}")  # Show first 10

#############################

# Detect and print outliers in TEST SET
print("\n=== Outliers (IQR Method) – Test Data ===")
outliers_iqr_test = detect_outliers_iqr(test_data)

for col, info in outliers_iqr_test.items():
    print(f"\nColumn: {col}")
    print(f"Number of outliers: {info['count']}")
    print(f"Percentage of outliers: {info['percentage']:.2f}%")
    if info['count'] > 0:
        print(f"Outlier values: {info['outlier_values'][:10]}")  # Show first 10



=== Outliers (IQR Method) – Train Data ===

Column: customer_id
Number of outliers: 0
Percentage of outliers: 0.00%

Column: age
Number of outliers: 0
Percentage of outliers: 0.00%

Column: membership_years
Number of outliers: 0
Percentage of outliers: 0.00%

Column: number_of_children
Number of outliers: 0
Percentage of outliers: 0.00%

Column: transaction_id
Number of outliers: 0
Percentage of outliers: 0.00%

Column: product_id
Number of outliers: 0
Percentage of outliers: 0.00%

Column: quantity
Number of outliers: 0
Percentage of outliers: 0.00%

Column: unit_price
Number of outliers: 0
Percentage of outliers: 0.00%

Column: discount_applied
Number of outliers: 0
Percentage of outliers: 0.00%

Column: transaction_hour
Number of outliers: 0
Percentage of outliers: 0.00%

Column: week_of_year
Number of outliers: 0
Percentage of outliers: 0.00%

Column: month_of_year
Number of outliers: 0
Percentage of outliers: 0.00%

Column: avg_purchase_value
Number of outliers: 0
Percentage of o

In [ ]:
#discount_sensitivity

'''
The original discount_sensitivity formula (total_discounts_received / total_items_purchased), which produces outliers like 79.81, 25.62, and 17.40, could be explained by customers receiving massive discounts during huge sales events (e.g., Black Friday, clearance sales, or bulk promotions), where high total_discounts_received values are divided by relatively low total_items_purchased, resulting in an inflated average discount per item.
'''

'\nThe original discount_sensitivity formula (total_discounts_received / total_items_purchased), which produces outliers like 79.81, 25.62, and 17.40, could be explained by customers receiving massive discounts during huge sales events (e.g., Black Friday, clearance sales, or bulk promotions), where high total_discounts_received values are divided by relatively low total_items_purchased, resulting in an inflated average discount per item.\n'

In [ ]:
# avg_transaction_value

'''
The extreme outliers in avg_transaction_value (e.g., 1518.80, 1177.00, 924.93), flagged as 11.54% of the data, can be explained by customers purchasing high-valued products (e.g., luxury goods, electronics, or bulk orders), where even a single transaction significantly exceeds typical purchase amounts. This naturally results in higher avg_transaction_value, contributing to the skewed distribution and emphasizing the disparity between standard and high-end purchasing behaviors.
'''

'\nThe extreme outliers in avg_transaction_value (e.g., 1518.80, 1177.00, 924.93), flagged as 11.54% of the data, can be explained by customers purchasing high-valued products (e.g., luxury goods, electronics, or bulk orders), where even a single transaction significantly exceeds typical purchase amounts. This naturally results in higher avg_transaction_value, contributing to the skewed distribution and emphasizing the disparity between standard and high-end purchasing behaviors.\n'

In [ ]:
#avg_discount_per_transaction

'''
The extreme outliers in avg_discount_per_transaction (e.g., 872.87, 177.08), flagged as 11.57% of the data, can be explained by customers purchasing high-valued products (e.g., luxury goods, electronics, or bulk orders), where even a moderate discount percentage (e.g., 10–20%) applied to a high avg_transaction_value (e.g., $8,000) results in a large absolute discount per transaction, inflating the average and contributing to the skewed distribution
'''

'\nThe extreme outliers in avg_discount_per_transaction (e.g., 872.87, 177.08), flagged as 11.57% of the data, can be explained by customers purchasing high-valued products (e.g., luxury goods, electronics, or bulk orders), where even a moderate discount percentage (e.g., 10–20%) applied to a high avg_transaction_value (e.g., $8,000) results in a large absolute discount per transaction, inflating the average and contributing to the skewed distribution\n'

In [ ]:
#total_return_rate

'''
The unusual outliers in total_return_rate, with 2,231 outliers (11.15%) and values like 0.42 (42%), 0.22 (22%), 0.18 (18%), 0.15 (15%), and 0.11 (11%), can be explained by customers returning items at a larger rate due to factors such as purchasing defective products, exploiting lenient return policies, or engaging in high-risk buying behaviors (e.g., buying multiple items to try and return), leading to elevated return rates that skew the distribution and produce a high percentage of outliers.
'''

'\nThe unusual outliers in total_return_rate, with 2,231 outliers (11.15%) and values like 0.42 (42%), 0.22 (22%), 0.18 (18%), 0.15 (15%), and 0.11 (11%), can be explained by customers returning items at a larger rate due to factors such as purchasing defective products, exploiting lenient return policies, or engaging in high-risk buying behaviors (e.g., buying multiple items to try and return), leading to elevated return rates that skew the distribution and produce a high percentage of outliers.\n'

In [ ]:
# Save train_data and test_data to CSV files
train_data.to_csv('SRS_train_data.csv', index=False)
test_data.to_csv('SRS_test_data.csv', index=False)

print("Train data saved to 'SRS_train_data.csv'")
print("Test data saved to 'SRS_test_data.csv'")

Train data saved to 'SRS_train_data.csv'
Test data saved to 'SRS_test_data.csv'
